In [10]:
from transitions import Machine

from transitions.extensions.asyncio import AsyncTimeout, AsyncMachine

import enum 
import logging
import asyncio

from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
from pprint import pprint

from datetime import datetime
from math import floor, ceil
import time
from time import sleep

import sys
import os
import zmq
import zmq.asyncio
# from s import Context
import json

logging.basicConfig(level=logging.INFO)

import ccxt
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
client = MongoClient()
db=client.fsmbot
trades = {}

context = zmq.asyncio.Context()


In [11]:
exchange = getattr(ccxt, 'binance')({'apikey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})

In [3]:
zContext = zmq.asyncio.Context()

class TradeModel(object):
    def __init__(self, trade): 
        self.trade = trade
        self.symbol = 'ALGOUSDT'
        
        # self.set_environment()
    def removeCurrentStopOnExchange(self, event):
        print('removing current stop on exchange...')

    def is_buy_signal(self, event):

        # print('is_buy_signal', event)
        ticker = event.kwargs['ticker']
        if float(ticker['curDayClose']) <= float(self.trade['trigger']):
            logging.info(f'+BUY triggered for {self.symbol} / {self.trade["_id"]}')
            # print(f'BUY triggered for {self.symbol} / {self.trade["_id"]}')
            return True
        logging.info(f'-BUY triggered for {self.symbol} / {self.trade["_id"]}')
        # print('trigger not tri/ggered')
        return False

    def buy_on_exchange(self, event):
        exchange.create_order(self.symbol, 'market', 'buy', self.trade["qty"]);
        
    
    def sellTriggered(self, event):
        True

    def machine_state_change(self, event):
        db.trades.update_one({'_id' : self.trade['_id'] }, {
        '$set': {
            'state': self.state
          }})

        # loop = asyncio.get_running_loop()
        # loop.run_until_complete(db.trades.update_one({'_id' : self.trade['_id'] }, {
        # '$set': {
        #     'state': self.state
        #   }}))
        # print(json.dumps(event))


# tradeModel = TradeModel(dbTrade)


states = [
    'ERROR',
    'DISABLED',
    'LIVE',
    'LONG',
    'STOPPED',
    'DONE'
]

# transitions = [['start', 'DISABLED', 'LIVE'],
#                ['disable', 'LIVE', 'DISABLED'],
#                ['buy', 'LIVE', 'LONG'],
#                ['movestop', 'LONG', 'LONG'],
#                ['stopped', 'LONG', 'STOPPED'],
#                ['sell', 'LONG', 'DONE'],
#                ['error', '*', 'ERROR']]
transitions = [
    { 'trigger': 'start', 'source': 'DISABLED', 'dest': 'LIVE'},
    { 'trigger': 'disable', 'source': 'LIVE', 'dest': 'DISABLED'},
    { 'trigger': 'tick', 'source': 'LIVE', 'dest': 'LONG', 'conditions': 'is_buy_signal', 'before' : 'buy_on_exchange'},
    { 'trigger': 'movestop', 'source': 'LONG', 'dest': 'LONG' ,'before': 'removeCurrentStopOnExchange'},
    { 'trigger': 'stopped', 'source': 'LONG', 'dest': 'STOPPED'},
    { 'trigger': 'tick', 'source': 'LONG', 'dest': 'DONE' ,'before': 'removeCurrentStopOnExchange', 'conditions': ['sellTriggered']},
    # { 'trigger': 'error', 'source': 'liquid', 'dest': 'gas', 'after': 'disappear' }
]

# async def runMachine():
#     machine = Machine(tradeModel, states=states, transitions=transitions, send_event=True, initial='DISABLED', after_state_change='machineStateChange')
#     tradeModel.start()
#     tradeModel.disable()
#     tradeModel.start()

# loop = asyncio.get_event_loop()
# loop.create_task(runMachine())

# await runMachine()
# asyncio.run(runMachine())
# tradeModel.buy()


# Lump now has state!
# lump.state

In [4]:
# loop = asyncio.get_running_loop()

In [5]:
# list(dbTrade)
for trade in db.trades.find():
    if not trade['symbol'] in trades: trades[trade['symbol']] = []
    tradeModel = TradeModel(trade)
    tradeMachine = Machine(tradeModel, states=states, transitions=transitions, send_event=True, initial='DISABLED', 
        after_state_change='machine_state_change')
    tradeMachine.set_state(trade['state'])
    trades[trade['symbol']].append({'machine': tradeMachine, 'model': tradeModel})

# trades
#     pass
# dbTrades = db.trades
# list(db.trades.find())




In [6]:


# Process 5 updates
async def run_ticker():
    socket = context.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        # print(symbol)
        if symbol in trades:        
            for trade in trades[symbol]:
                model = trade['model']
                # [tradeMachine, tradeModel] = trade
                # print(trade['model'])
                if model.state in ['LIVE', 'LONG']:
                    # print(ticker)
                    # print(symbol)
                    model.tick(ticker=ticker)
        # pprint(ticker)
        # await asyncio.sleep(0.01)
    
await run_ticker()
    # print('received', ticker)
    # zipcode, temperature, relhumidity = string.split()
    # total_temp += int(temperature)
# loop = asyncio.get_event_loop()
# loop.create_task(run_ticker())


INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:+BUY triggered for ALGOUSDT / 615cc81fa0d373f6b44b88b0
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_change'
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1
INFO:root:-BUY triggered for ALGOUSDT / 615d033da0d373f6b44b88b1


CancelledError: 

In [7]:
# trades['SOLUSDT'][0]['model'].disable()
model = trades['ALGOUSDT'][0]['model']
# trades['ALGOUSDT'][0]['model'].tick(ticker={'curDayClose': 1})
# loop = asyncio.get_event_loop()
# loop.create_task(model.tick(ticker={'curDayClose': 1}))
# asyncio.gather([model.tick(ticker={'curDayClose': 1})])
# asyncio.get_event_loop().run_until_complete()

# asyncio.run(asyncio.wait([model.tick(ticker={'curDayClose': 1})]))
# await asyncio.wait(model.tick(ticker={'curDayClose': 1}))
model.tick(ticker={'curDayClose': "1"})
# loop = asyncio.get_event_loop()
# loop.run_until_complete()
# model.start()
# model.disable()
trades['ALGOUSDT'][0]['model'].state
# .start()

INFO:root:BUY triggered for ALGOUSDT / 615cc81fa0d373f6b44b88b0
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_change'


is_buy_signal <EventData('<State('LIVE')@140700378186032>', <Transition('LIVE', 'LONG')@140700377665696>)@140700378188576>
BUY triggered for ALGOUSDT / 615cc81fa0d373f6b44b88b0


'LONG'

In [ ]:
float("")